In [37]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')


from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn import preprocessing
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error, mean_squared_log_error
import matplotlib.pyplot as plt
%matplotlib inline
df = pd.read_csv('final_flight_data')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
df.head()

,Month,DayofMonth,Hour,Origin,Dest,SchedDep,DepTime,DepDelay,ArrTime,ArrDelay,AirTime,FlightNumber,Temp,Humidity,Wind,Pressure,Description,Delay_min
0,1,1,22,DFW,MIA,1900-01-01 21:20:00,1900-01-01 22:04:00,84.0,139.0,73.0,136.0,AA2329,65.01,77.0,2.0,1009.0,broken clouds,44.0
1,1,1,22,DFW,LAX,1900-01-01 22:10:00,1900-01-01 22:16:00,6.0,2353.0,9.0,184.0,AA1023,65.01,77.0,2.0,1009.0,broken clouds,6.0
2,1,1,22,DFW,ORD,1900-01-01 21:30:00,1900-01-01 22:20:00,90.0,19.0,74.0,98.0,AA1040,65.01,77.0,2.0,1009.0,broken clouds,50.0
3,1,1,22,DFW,ORD,1900-01-01 22:15:00,1900-01-01 22:08:00,-7.0,2400.0,-30.0,95.0,AA2525,65.01,77.0,2.0,1009.0,broken clouds,-7.0
4,1,1,18,LAX,ORD,1900-01-01 17:50:00,1900-01-01 18:37:00,87.0,21.0,58.0,200.0,AA2576,47.59,81.0,3.0,1014.0,fog,47.0


In [3]:
df['Origin'].nunique()

5

In [4]:
df.columns

Index(['Month', 'DayofMonth', 'Hour', 'Origin', 'Dest', 'SchedDep', 'DepTime',
       'DepDelay', 'ArrTime', 'ArrDelay', 'AirTime', 'FlightNumber', 'Temp',
       'Humidity', 'Wind', 'Pressure', 'Description', 'Delay_min'],
      dtype='object')

In [5]:
df_reg = df[['Temp','Humidity','Wind','Pressure','Month','Delay_min','Origin','Dest', 'ArrDelay']]

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 66180 entries, 0 to 66179
Data columns (total 18 columns):
Month           66180 non-null int64
DayofMonth      66180 non-null int64
Hour            66180 non-null int64
Origin          66180 non-null object
Dest            66180 non-null object
SchedDep        66180 non-null object
DepTime         66180 non-null object
DepDelay        66180 non-null float64
ArrTime         66180 non-null float64
ArrDelay        66180 non-null float64
AirTime         66180 non-null float64
FlightNumber    66180 non-null object
Temp            66180 non-null float64
Humidity        66180 non-null float64
Wind            66180 non-null float64
Pressure        66180 non-null float64
Description     66180 non-null object
Delay_min       66180 non-null float64
dtypes: float64(9), int64(3), object(6)
memory usage: 9.1+ MB


In [7]:
df.corr()

,Month,DayofMonth,Hour,DepDelay,ArrTime,ArrDelay,AirTime,Temp,Humidity,Wind,Pressure,Delay_min
Month,1.000000,-0.015237,0.013013,-0.023012,-0.006746,-0.021540,-0.011686,0.541680,-0.065682,-0.156282,-0.031801,-0.028550
DayofMonth,-0.015237,1.000000,-0.000209,-0.019401,-0.001348,-0.022907,-0.003865,0.057572,-0.008947,0.003257,-0.100276,-0.021099
Hour,0.013013,-0.000209,1.000000,0.125619,0.193614,0.105412,0.035982,0.210621,-0.353273,0.239275,0.024344,0.126451
DepDelay,-0.023012,-0.019401,0.125619,1.000000,-0.020411,0.921155,0.021688,0.025830,-0.014603,0.041354,-0.019178,0.982366
ArrTime,-0.006746,-0.001348,0.193614,-0.020411,1.000000,-0.017324,-0.038889,-0.038885,0.013304,-0.007182,0.044226,-0.015902
ArrDelay,-0.021540,-0.022907,0.105412,0.921155,-0.017324,1.000000,0.035785,0.032528,-0.008624,0.054609,-0.033638,0.904374
AirTime,-0.011686,-0.003865,0.035982,0.021688,-0.038889,0.035785,1.000000,-0.077090,-0.000613,-0.174676,-0.037591,0.030301
Temp,0.541680,0.057572,0.210621,0.025830,-0.038885,0.032528,-0.077090,1.000000,-0.235722,0.027813,-0.102388,0.022648
Humidity,-0.065682,-0.008947,-0.353273,-0.014603,0.013304,-0.008624,-0.000613,-0.235722,1.000000,-0.244963,-0.149366,-0.013771
Wind,-0.156282,0.003257,0.239275,0.041354,-0.007182,0.054609,-0.174676,0.027813,-0.244963,1.000000,0.007175,0.041775


For the continuous features of the dataset to predict the target variable delay minutes

In [8]:
# Create X and y then split in train and test

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
y = df_reg['ArrDelay']
X = df_reg[['Temp', 'Humidity', 'Wind', 'Pressure','Delay_min']]

In [34]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=100)


Performing our first linear regression modelling

In [39]:
from sklearn.metrics import mean_squared_error, mean_squared_log_error
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import Imputer


# Fit the model and print R2 and MSE for train and test
linreg = LinearRegression()
linreg.fit(X_train, y_train)

print('Training r^2:', linreg.score(X_train, y_train))
print('Testing r^2:', linreg.score(X_test, y_test))
print('Training MSE:', mean_squared_error(y_train, linreg.predict(X_train)))
print('Testing MSE:', mean_squared_error(y_test, linreg.predict(X_test)))

Training r^2: 0.8231983200757722
Testing r^2: 0.804161213898475
Training MSE: 365.12831478845317
Testing MSE: 383.04083687580703


In [11]:
from sklearn.preprocessing import StandardScaler

# Scale the train and test data
ss = StandardScaler()
ss.fit(X_train)

X_train_scaled = ss.transform(X_train)
X_test_scaled = ss.transform(X_test)

In [12]:
linreg_norm = LinearRegression()
linreg_norm.fit(X_train_scaled, y_train)

print('Scaled Training r^2:', linreg_norm.score(X_train_scaled, y_train))
print('Scaled Testing r^2:', linreg_norm.score(X_test_scaled, y_test))
print('Scaled Training MSE:', mean_squared_error(y_train, linreg_norm.predict(X_train_scaled)))
print('Scaled Testing MSE:', mean_squared_error(y_test, linreg_norm.predict(X_test_scaled)))

Scaled Training r^2: 0.8231983200757722
Scaled Testing r^2: 0.804161213898475
Scaled Training MSE: 365.1283147884533
Scaled Testing MSE: 383.04083687580703


In [13]:
X_cat = df_reg[['Month', 'Origin', 'Dest']]


In [14]:
X_train_cat, X_test_cat, y_train, y_test = train_test_split(X_cat, y,test_size=0.25, random_state=100)

In [15]:
from sklearn.preprocessing import OneHotEncoder

# OneHotEncode Categorical variables
ohe = OneHotEncoder(handle_unknown='ignore')
ohe.fit(X_train_cat)

X_train_ohe = ohe.transform(X_train_cat)
X_test_ohe = ohe.transform(X_test_cat)

columns = ohe.get_feature_names(input_features=X_train_cat.columns)
cat_train_df = pd.DataFrame(X_train_ohe.todense(), columns=columns)
cat_test_df = pd.DataFrame(X_test_ohe.todense(), columns=columns)

In [16]:
X_train_all = pd.concat([pd.DataFrame(X_train_scaled), cat_train_df], axis = 1)
X_test_all = pd.concat([pd.DataFrame(X_test_scaled), cat_test_df], axis = 1)

In [17]:
linreg_all = LinearRegression()
linreg_all.fit(X_train_all, y_train)

print('Continuous and Categorical')
print('Training r^2:', linreg_all.score(X_train_all, y_train))
print('Testing r^2:', linreg_all.score(X_test_all, y_test))
print('Training MSE:', mean_squared_error(y_train, linreg_all.predict(X_train_all)))
print('Testing MSE:', mean_squared_error(y_test, linreg_all.predict(X_test_all)))

Continuous and Categorical
Training r^2: 0.8248431672803317
Testing r^2: 0.8058559330704745
Training MSE: 361.7313997357077
Testing MSE: 379.72613776622575


In [18]:
from sklearn.linear_model import Lasso

lasso = Lasso() #Lasso is also known as the L1 norm.
lasso.fit(X_train_all, y_train)
print( 'Lasso')
print('Training r^2:', lasso.score(X_train_all, y_train))
print('Testing r^2:', lasso.score(X_test_all, y_test))
print('Training MSE:', mean_squared_error(y_train, lasso.predict(X_train_all)))
print('Testing MSE:', mean_squared_error(y_test, lasso.predict(X_test_all)))

Lasso
Training r^2: 0.8219976164783861
Testing r^2: 0.8028705906449709
Training MSE: 367.60799078057005
Testing MSE: 385.56516528365006


In [19]:
lasso = Lasso(10) #Lasso is also known as the L1 norm.
lasso.fit(X_train_all, y_train)
print('Lasso a=10')
print('Training r^2:', lasso.score(X_train_all, y_train))
print('Testing r^2:', lasso.score(X_test_all, y_test))
print('Training MSE:', mean_squared_error(y_train, lasso.predict(X_train_all)))
print('Testing MSE:', mean_squared_error(y_test, lasso.predict(X_test_all)))

Lasso a=10
Training r^2: 0.774060040553496
Testing r^2: 0.7562418221902506
Training MSE: 466.6079907805704
Testing MSE: 476.76631520359007


In [20]:
from sklearn.linear_model import Ridge

ridge = Ridge() #Ridge is also known as the L2 norm.
ridge.fit(X_train_all, y_train)
print('Ridge')
print('Training r^2:', ridge.score(X_train_all, y_train))
print('Testing r^2:', ridge.score(X_test_all, y_test))
print('Training MSE:', mean_squared_error(y_train, ridge.predict(X_train_all)))
print('Testing MSE:', mean_squared_error(y_test, ridge.predict(X_test_all)))

Ridge
Training r^2: 0.8248431669039311
Testing r^2: 0.8058559717110396
Training MSE: 361.7314005130449
Testing MSE: 379.72606218919293


In [21]:
from sklearn.linear_model import Ridge

ridge = Ridge(alpha = 10) #Lasso is also known as the L1 norm.
ridge.fit(X_train_all, y_train)
print('Ridge')
print('Training r^2:', ridge.score(X_train_all, y_train))
print('Testing r^2:', ridge.score(X_test_all, y_test))
print('Training MSE:', mean_squared_error(y_train, ridge.predict(X_train_all)))
print('Testing MSE:', mean_squared_error(y_test, ridge.predict(X_test_all)))

Ridge
Training r^2: 0.8248431296677473
Testing r^2: 0.8058562853224569
Training MSE: 361.731477412682
Testing MSE: 379.72544879701377


In [22]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

In [23]:
for index, degree in enumerate([2, 3, 4,]):
    
    # Instantiate PolynomialFeatures
    poly = PolynomialFeatures(degree)
    
    # Fit and transform X_train
    X_poly_train = poly.fit_transform(X_train)
    
    # Instantiate and fit a linear regression model to the polynomial transformed train features
    reg_poly = LinearRegression().fit(X_poly_train, y_train)
    

#     X_train_scaled = ss.transform(X_train)
#     X_test_scaled = ss.transform(X_test)
    
    # Transform the test data into polynomial features
    X_poly_test = poly.transform(X_test)
    
    ss.fit(X_train)

    # Get predicted values for transformed polynomial test data  
    y_pred = reg_poly.predict(X_poly_test)
    
    # Evaluate model performance on test data
    print("degree %d" % degree, r2_score(y_test, y_pred))
    
 
    



degree 2 0.8078956644436939
degree 3 0.8108793365978026
degree 4 0.8098774587554716


In [24]:
    print('Training r^2:', reg_poly.score(X_poly_train, y_train))
    print('Testing r^2:',  reg_poly.score(X_poly_test, y_test))
    print('Training MSE:', mean_squared_error(y_train, reg_poly.predict(X_poly_train)))
    print('Testing MSE:', mean_squared_error(y_test, reg_poly.predict(X_poly_test)))

Training r^2: 0.8332264955701296
Testing r^2: 0.8098774587554716
Training MSE: 344.4182693848871
Testing MSE: 371.86044070711387


In [25]:
import statsmodels.api as sm

X_1 = sm.add_constant(X)
#Fitting sm.OLS model
model = sm.OLS(y,X_1).fit()
model.pvalues

const        1.674501e-13
Temp         1.459941e-12
Humidity     3.349147e-07
Wind         2.734163e-28
Pressure     4.527938e-18
Delay_min    0.000000e+00
dtype: float64

In [26]:
#Backward Elimination
cols = list(X.columns)
pmax = 1
while (len(cols)>0):
    p= []
    X_1 = X[cols]
    X_1 = sm.add_constant(X_1)
    model = sm.OLS(y,X_1).fit()
    p = pd.Series(model.pvalues.values[1:],index = cols)      
    pmax = max(p)
    feature_with_p_max = p.idxmax()
    if(pmax>0.05):
        cols.remove(feature_with_p_max)
    else:
        break
selected_features_BE = cols
print(selected_features_BE)

['Temp', 'Humidity', 'Wind', 'Pressure', 'Delay_min']


In [27]:
from sklearn.feature_selection import RFE
model = LinearRegression()
#Initializing RFE model
rfe = RFE(model, 7)
#Transforming data using RFE
X_rfe = rfe.fit_transform(X,y)  
#Fitting the data to model
model.fit(X_rfe,y)
print(rfe.support_)
print(rfe.ranking_)

#no of features
nof_list=np.arange(1,13)            
high_score=0
#Variable to store the optimum features
nof=0           
score_list =[]
for n in range(len(nof_list)):
    X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.25, random_state = 100)
    model = LinearRegression()
    rfe = RFE(model,nof_list[n])
    X_train_rfe = rfe.fit_transform(X_train,y_train)
    X_test_rfe = rfe.transform(X_test)
    model.fit(X_train_rfe,y_train)
    score = model.score(X_test_rfe,y_test)
    score_list.append(score)
    if(score>high_score):
        high_score = score
        nof = nof_list[n]
print("Optimum number of features: %d" %nof)
print("Score with %d features: %f" % (nof, high_score))


[ True  True  True  True  True]
[1 1 1 1 1]
Optimum number of features: 5
Score with 5 features: 0.804161


In [28]:
# 10-fold cross-validation with all three features
# instantiate model
lm = LinearRegression()

# store scores in scores object
# we can't use accuracy as our evaluation metric since that's only relevant for classification problems
# RMSE is not directly available so we will use MSE
scores = cross_val_score(lm, X_train, y_train, cv=10, scoring='r2')
mse_scores = cross_val_score(lm, X, y, cv=10, scoring='neg_mean_squared_error')
print('Mean r2:',np.mean(scores))
print('Mean r2:',np.mean(scores))
print('10 Fold Score:',scores)
print ('10 Fold mean squared error',-(mse_scores) )



Mean r2: 0.8197954074600753
Mean r2: 0.8197954074600753
10 Fold Score: [0.74284105 0.83405038 0.83256317 0.81403775 0.83767335 0.84898614
 0.81541739 0.83837405 0.82137993 0.81263086]
10 Fold mean squared error [435.59090534 383.65409378 350.65261708 373.31696311 403.66176261
 405.53992648 431.01082107 378.08429521 266.73286138 278.2949196 ]


In [31]:
from reg import create_Linear_regression_model


In [40]:
create_Linear_regression_model(X_train, X_test, y_train, y_test)

Training r^2: 0.8231983200757722
Testing r^2: 0.804161213898475
Training MSE: 365.12831478845317
Testing MSE: 383.04083687580703


In [55]:
from reg import create_model


In [60]:
create_model(X_train, X_test, y_train, y_test,2)

Training r^2: 0.8231983200757722
Testing r^2: 0.804161213898475
Training MSE: 365.12831478845317
Testing MSE: 383.04083687580703
The model performance for the training set
-------------------------------------------
RMSE of training set is 18.884889586531404
R2 score of training set is 0.8273089720985292


The model performance for the test set
-------------------------------------------
RMSE of test set is 19.3839270975634
R2 score of test set is 0.8078956644436939
